In [9]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mahotas

In [ ]:



# Load the image
image_path = "/home/duyle/Rice_photos/BC-15/BC-15/DSC7436_idx47.png"  # Update with your image path
image = cv2.imread(image_path)

# Convert to grayscale
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur to reduce noise
blur = cv2.GaussianBlur(image, (3, 3), 0)
threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = max(contours, key=cv2.contourArea)


    # Compute Hu Moments
moments = cv2.moments(contour)
hu_moments = cv2.HuMoments(moments)

# Apply log transformation with a small epsilon to prevent log(0) issues
hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)


print(hu_moments)

[[ 0.54814873]
 [ 1.26453107]
 [ 4.13240974]
 [ 4.58620361]
 [ 8.91317259]
 [ 5.25173079]
 [-9.57361252]]


In [199]:
from scipy.ndimage import convolve
def extract_edge_histogram_features(image):
    """
    Compute a simplified edge histogram descriptor (EDH) from the image.
    The function applies five convolution kernels corresponding to different edge orientations.
    """
    # Convert to grayscale if necessary.
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Define convolution kernels for five edge types.
    kernels = {
        "vertical": np.array([[-1,  2, -1],
                              [-1,  2, -1],
                              [-1,  2, -1]], dtype=np.float32),
        "horizontal": np.array([[-1, -1, -1],
                                [ 2,  2,  2],
                                [-1, -1, -1]], dtype=np.float32),
        "diag_45": np.array([[-1, -1,  2],
                             [-1,  2, -1],
                             [ 2, -1, -1]], dtype=np.float32),
        "diag_135": np.array([[ 2, -1, -1],
                              [-1,  2, -1],
                              [-1, -1,  2]], dtype=np.float32),
        # A non-directional (Laplacian-like) kernel.
        "non_directional": np.array([[1,  1,  1],
                                     [1, -8,  1],
                                     [1,  1,  1]], dtype=np.float32)
    }
    
    features = {}
    total_energy = 0.0
    
    # Convolve the image with each kernel and compute the energy.
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_{key}"] = energy
        total_energy += energy
    
    # Normalize to form a histogram (if total energy > 0).
    if total_energy > 0:
        for key in list(features.keys()):
            features[key] /= total_energy

    return features

In [201]:
extract_edge_histogram_features(cv2.imread('/home/duyle/Rice_photos/BC-15/BC-15/DSC7436_idx70.png'))

{'edge_energy_vertical': np.float32(0.11537549),
 'edge_energy_horizontal': np.float32(0.246982),
 'edge_energy_diag_45': np.float32(0.16055416),
 'edge_energy_diag_135': np.float32(0.15892461),
 'edge_energy_non_directional': np.float32(0.31816378)}

In [5]:
def color_structure_descriptor(image, grid_size=(8, 8), bins=32):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    
    height, width = hsv_image.shape[:2]
    cell_h, cell_w = height // grid_size[0], width // grid_size[1]

    
    hist_bins = [bins] 

    
    csd = []

    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            x_start, y_start = col * cell_w, row * cell_h
            x_end, y_end = x_start + cell_w, y_start + cell_h
            cell = hsv_image[y_start:y_end, x_start:x_end]

            hist = cv2.calcHist([cell], [0], None, hist_bins, [0, 180])
            hist = hist.flatten() 
            hist = hist / hist.sum() if hist.sum() != 0 else hist

            csd.extend(hist)

    return {f"csd_{i}":csd[i] for i in range(len(csd))}


In [3]:
image_path = "/home/duyle/Rice_photos/BC-15/BC-15/DSC7436_idx47.png"  # Update with your image path
image = cv2.imread(image_path)
dic=  color_structure_descriptor(image)


In [4]:
np.set_printoptions(threshold=np.inf)  # Displays the full array
dic

{'color_0': np.float32(0.0),
 'color_1': np.float32(0.0),
 'color_2': np.float32(0.0),
 'color_3': np.float32(0.0074074073),
 'color_4': np.float32(0.94814813),
 'color_5': np.float32(0.044444446),
 'color_6': np.float32(0.0),
 'color_7': np.float32(0.0),
 'color_8': np.float32(0.0),
 'color_9': np.float32(0.0),
 'color_10': np.float32(0.0),
 'color_11': np.float32(0.0),
 'color_12': np.float32(0.0),
 'color_13': np.float32(0.0),
 'color_14': np.float32(0.0),
 'color_15': np.float32(0.0),
 'color_16': np.float32(0.0),
 'color_17': np.float32(0.0),
 'color_18': np.float32(0.0),
 'color_19': np.float32(0.0),
 'color_20': np.float32(0.0),
 'color_21': np.float32(0.0),
 'color_22': np.float32(0.0),
 'color_23': np.float32(0.0),
 'color_24': np.float32(0.0),
 'color_25': np.float32(0.0),
 'color_26': np.float32(0.0),
 'color_27': np.float32(0.0),
 'color_28': np.float32(0.0),
 'color_29': np.float32(0.0),
 'color_30': np.float32(0.0),
 'color_31': np.float32(0.0),
 'color_32': np.float32(0.